In [1]:
import torch
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

from google.colab import drive
drive.mount('/content/drive')


# Create a persistent project directory
import os
PROJECT_DIR = '/content/drive/MyDrive/AI574_Multi_Domain_Agent'
os.makedirs(PROJECT_DIR, exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/data', exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/models', exist_ok=True)
print(f'Project directory: {PROJECT_DIR}')


CUDA available: True
GPU: NVIDIA A100-SXM4-80GB
Memory: 85.2 GB
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Project directory: /content/drive/MyDrive/AI574_Multi_Domain_Agent


In [2]:
# # Action: Run this cell to install all packages:

# # %%capture
# !pip install -q langchain>=0.3.0 langchain-core>=0.3.0 langgraph>=0.2.0 langchain-text-splitters>=0.2.0
# !pip install -q keras>=3.0 keras-hub>=0.17.0
# !pip install -q jax[cuda12] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
# !pip install -q sentence-transformers>=3.0.0
# !pip install -q chromadb>=0.5.0
# !pip install -q PyPDF2>=3.0.0 arxiv>=2.1.0 pydantic>=2.0

In [3]:
import os
os.environ['KERAS_BACKEND'] = 'jax'

# Verify
import keras
print(f'Keras backend: {keras.backend.backend()}')
print(f'Keras version: {keras.__version__}')

Keras backend: jax
Keras version: 3.10.0


In [4]:
import langchain
import langgraph
import keras_hub
import chromadb
from sentence_transformers import SentenceTransformer
import jax
print('All imports successful!')
print(f'JAX devices: {jax.devices()}')

All imports successful!
JAX devices: [CudaDevice(id=0)]


In [5]:
import os
import sys

# PROJECT_DIR is already defined
PROJECT_ROOT = os.path.join(PROJECT_DIR, 'project')

dirs = [
    'config/prompts',
    'foundation',
    'ingestion',
    'rag_core',
    'agents',
    'orchestration',
    'evaluation',
    'data/industrial',
    'data/recipes',
]

# Create directories
for d in dirs:
    os.makedirs(os.path.join(PROJECT_ROOT, d), exist_ok=True)

# Create __init__.py in every package directory
packages = [
    'config',
    'foundation',
    'ingestion',
    'rag_core',
    'agents',
    'orchestration',
    'evaluation'
]

for pkg in packages:
    init_file = os.path.join(PROJECT_ROOT, pkg, '__init__.py')
    open(init_file, 'a').close()

# Add project root to Python path
sys.path.insert(0, PROJECT_ROOT)

print('Project structure created at:', PROJECT_ROOT)

Project structure created at: /content/drive/MyDrive/AI574_Multi_Domain_Agent/project


In [6]:
import os, sys

# PROJECT_DIR already defined elsewhere
PROJECT_ROOT = os.path.join(PROJECT_DIR, "project")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT =", PROJECT_ROOT)
print("foundation exists?", os.path.isdir(os.path.join(PROJECT_ROOT, "foundation")))
print("init exists?", os.path.isfile(os.path.join(PROJECT_ROOT, "foundation", "__init__.py")))
print("files:", os.listdir(os.path.join(PROJECT_ROOT, "foundation")))

PROJECT_ROOT = /content/drive/MyDrive/AI574_Multi_Domain_Agent/project
foundation exists? True
init exists? True
files: ['__init__.py', 'llm_wrapper.py', '__pycache__', 'embedding_service.py', 'vector_store.py']


In [7]:
import os, sys
from config.settings import CONFIG, LLMConfig
from foundation.llm_wrapper import KerasHubChatModel
from foundation.embedding_service import EmbeddingService
from foundation.vector_store import VectorStoreService

# Print configuration
print(f"LLM: {CONFIG.llm.preset}")
print(f"Embeddings: {CONFIG.embedding.model_name}")
print(f"Domains: {CONFIG.supervisor.domains}")

from google.colab import userdata
import os

os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

LLM: gemma2_instruct_9b_en
Embeddings: Alibaba-NLP/gte-large-en-v1.5
Domains: ['industrial', 'recipe', 'scientific']


In [8]:
# Now try loading
from foundation.llm_wrapper import KerasHubChatModel
from config.settings import CONFIG

llm = KerasHubChatModel(config=CONFIG.llm)
print("✅ LLM loaded")


✅ LLM loaded


In [9]:
from foundation.embedding_service import EmbeddingService
from config.settings import EmbeddingConfig

embed_config = EmbeddingConfig(
    model_name="thenlper/gte-large",   # v1 — stable, no custom code
    dimension=1024,
    device="cpu",
    trust_remote_code=False,
)
embedder = EmbeddingService(config=embed_config)

test_vec = embedder.embed_query("PLC fault code")
print(f"✅ Embedding dimension: {len(test_vec)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]

BertModel LOAD REPORT from: thenlper/gte-large
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Embedding dimension: 1024


In [10]:
from foundation.vector_store import VectorStoreService

vs = VectorStoreService(embedding_service=embedder)
print("✅ Vector store initialized")
print(vs.get_all_stats())

✅ Vector store initialized
[{'domain': 'industrial', 'collection_name': 'industrial_knowledge', 'document_count': 0}, {'domain': 'recipe', 'collection_name': 'recipe_knowledge', 'document_count': 0}, {'domain': 'scientific', 'collection_name': 'scientific_knowledge', 'document_count': 0}]


In [11]:
from ingestion.index_builder import IndexBuilder
from ingestion.document_loader import DocumentLoader
from ingestion.chunking_pipeline import ChunkingPipeline

builder = IndexBuilder(vector_store=vs)

# ── Industrial: Rockwell Automation / Allen-Bradley ──
sample_industrial = [
    """Fault Code F002 - Allen-Bradley PowerFlex 525 VFD: Auxiliary Input.
    Description: An external fault has been detected via the auxiliary input terminal.
    Possible Causes:
    1. External device connected to auxiliary input is reporting a fault condition
    2. Wiring issue on the auxiliary digital input terminal
    3. Incorrect parameter configuration for the auxiliary input function
    Troubleshooting Steps:
    1. Check the external device connected to the auxiliary input for faults
    2. Verify wiring connections on terminal block TB2
    3. Check parameter A070 [Fault Config 2] for correct auxiliary input configuration
    4. Inspect for damaged or loose wiring on the digital input terminals
    Resolution: Clear the external fault condition and reset the drive by cycling the Enable input or pressing the Stop/Reset key on the HIM.""",

    """Fault Code F004 - Allen-Bradley PowerFlex 525 VFD: Undervoltage.
    Description: The DC bus voltage has dropped below the undervoltage trip level.
    Possible Causes:
    1. Input power supply voltage too low or momentary power dip
    2. Incoming power supply fuses blown or circuit breaker tripped
    3. Loose connections on input power terminals R/L1, S/L2, T/L3
    4. DC bus capacitors degraded (check capacitor health indicator LED)
    Troubleshooting Steps:
    1. Measure incoming line voltage at drive terminals - must be within nameplate rating +/-10%
    2. Check all three phases for voltage balance (max 3% imbalance)
    3. Inspect and tighten input power terminal connections (torque to 1.4 Nm)
    4. Check parameter A531 [DC Bus Voltage] for current reading
    5. Review fault log in parameter A700-A706 for fault history and timestamps
    Resolution: Restore proper input voltage. If capacitors are degraded, replace the drive.
    SAFETY: Disconnect and lockout/tagout all power sources before inspecting terminals.
    Wait 5 minutes for DC bus capacitors to discharge before servicing.""",

    """Fault Code F005 - Allen-Bradley PowerFlex 525 VFD: Overvoltage.
    Description: The DC bus voltage has exceeded the overvoltage trip level.
    Possible Causes:
    1. Input line voltage exceeds drive nameplate rating
    2. Excessive regenerative energy from motor deceleration (overhauling load)
    3. Deceleration time too short for the load inertia
    4. Dynamic brake resistor circuit failure or incorrect sizing
    Troubleshooting Steps:
    1. Measure input voltage - must not exceed 528 VAC for 480V class drives
    2. Increase deceleration time in parameter A092 [Decel Time 1]
    3. Enable bus regulator: set parameter A540 [Bus Reg Mode] to option 1 (Enabled)
    4. If using dynamic braking, check DB resistor connections and resistance value
    5. Check parameter A531 [DC Bus Voltage] - nominal is ~650V for 480V input
    Resolution: Reduce input voltage or extend deceleration time. Add DB resistor for overhauling loads.""",

    """Allen-Bradley CompactLogix 5380 Controller - EtherNet/IP Configuration Guide.
    Model: 1769-L33ER CompactLogix 5380
    Prerequisites: RSLogix 5000 v32+ or Studio 5000 Logix Designer v32+
    Step 1: Create new project - select 1769-L33ER controller, revision 32+
    Step 2: Configure controller IP address via USB connection using BOOTP/DHCP tool
    Step 3: Add EtherNet/IP module in I/O Configuration tree
    Step 4: Right-click controller > Properties > set IP address (e.g., 192.168.1.10)
    Step 5: Add remote I/O devices - right-click EtherNet/IP > New Module
    Step 6: Configure Produced/Consumed tags for controller-to-controller communication
    Step 7: Download program and verify connection indicators:
      - OK LED: Solid green = running, Flashing green = program mode
      - ENET LED: Solid green = has connections, Flashing green = no connections
    Common Issue: If ENET LED is off, check IP config and network switch connection.
    Use RSLinx Classic > RSWho to verify controller appears on EtherNet/IP network.""",

    """Allen-Bradley PanelView Plus 7 HMI - Troubleshooting Communication Failures.
    Symptom: PanelView displays "Controller not found" or shows stale data.
    Possible Causes:
    1. Incorrect IP address or subnet mask configuration
    2. EtherNet/IP cable disconnected or faulty
    3. RSLinx Enterprise communication path misconfigured
    4. Controller is in Program mode instead of Run mode
    Troubleshooting Steps:
    1. On PanelView: press Terminal Settings > Networks > verify IP address
    2. Ping the controller IP from PanelView diagnostics screen
    3. In FactoryTalk View Studio: check Communication Setup > verify shortcut path
    4. Ensure controller and HMI are on same subnet (e.g., both 192.168.1.x/24)
    5. Check Ethernet cable and switch port LEDs for link activity
    6. Review controller mode - must be in Run or Remote Run for live data
    Resolution: Correct IP addressing and verify physical network connectivity.
    If using managed switch, verify VLANs are not isolating devices.""",

    """Preventive Maintenance Schedule - Allen-Bradley PowerFlex 755 Drive System.
    Weekly: Check drive status LEDs and HIM display for active alarms
    Monthly: Inspect cooling fans for proper operation, clean air filters
    Quarterly: Verify DC bus voltage (parameter 15 - Bus Voltage), check capacitor
    formation indicator, tighten all power and control terminal connections
    Semi-Annually: Thermal scan of power connections, verify ground fault monitoring,
    backup drive parameters using Connected Components Workbench (CCW)
    Annually: Full parameter backup, firmware version audit against Rockwell
    compatibility matrix, capacitor health test, clean power module heat sinks
    Every 5 Years: Replace cooling fans (recommended life), capacitor reformation
    MTBF: PowerFlex 755 rated at approximately 200,000 hours (28+ years) at 40C ambient.
    Critical Spares to Stock: Cooling fan assembly (KIT-F755FAN), control board fuse,
    fiber optic cables for multi-axis configurations.""",
]

count = builder.index_industrial_texts(sample_industrial, source="rockwell_automation")
print(f"✅ Indexed {count} industrial chunks")

# ── Recipe sample data ──
sample_recipes = [
    """Recipe: Classic Chocolate Chip Cookies
    Prep Time: 45 minutes. Ingredients: 2 1/4 cups flour, 1 tsp baking soda,
    1 tsp salt, 1 cup butter softened, 3/4 cup sugar, 3/4 cup brown sugar,
    2 large eggs, 2 tsp vanilla, 2 cups chocolate chips.
    Egg Substitutes: Use 1/4 cup unsweetened applesauce per egg,
    or 1 mashed banana per egg, or 3 tbsp aquafaba per egg.""",

    """Recipe: Quick Chicken Stir-Fry
    Prep Time: 20 minutes. Ingredients: 1 lb chicken breast, 2 bell peppers,
    1 cup rice, 3 tbsp soy sauce, 1 tbsp sesame oil, 2 cloves garlic, ginger.
    Steps: Cook rice. Slice chicken. Heat oil in wok over high heat.
    Stir-fry chicken 5-6 min. Add vegetables and garlic. Add soy sauce.
    Nutrition: ~450 calories per serving, 35g protein.""",

    """Recipe: Homemade Margherita Pizza
    Prep Time: 90 minutes (including dough rise). Ingredients: 3 cups flour,
    1 packet yeast, 1 cup warm water, 2 tbsp olive oil, 1 tsp salt, 1 tsp sugar,
    1 cup crushed San Marzano tomatoes, 8 oz fresh mozzarella, fresh basil.
    Steps: Mix dough, let rise 1 hour. Preheat oven to 475F. Stretch dough,
    add sauce and cheese. Bake 12-15 min until crust is golden.
    Gluten-Free Alternative: Substitute 3 cups gluten-free flour blend plus
    1 tsp xanthan gum for regular flour.""",
]

recipe_docs = DocumentLoader.from_texts(sample_recipes, "recipe", "demo_recipes")
chunker = ChunkingPipeline()
recipe_chunks = chunker.chunk_documents(recipe_docs, domain="recipe")
count = vs.add_documents("recipe", recipe_chunks)
print(f"✅ Indexed {count} recipe chunks")

# ── Scientific sample data ──
sample_scientific = [
    """Title: Attention Is All You Need
    Authors: Vaswani, Shazeer, Parmar, Uszkoreit, Jones, Gomez, Kaiser, Polosukhin
    Abstract: We propose a new network architecture, the Transformer, based solely
    on attention mechanisms, dispensing with recurrence and convolutions entirely.
    The Transformer allows for significantly more parallelization and achieves
    new state of the art in translation quality. ArXiv ID: 1706.03762""",

    """Title: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
    Authors: Lewis, Perez, Piktus, Petroni, Karpukhin, Goyal, Kuttler, Lewis, Yih, Rocktaschel, Riedel, Kiela
    Abstract: We explore a general-purpose fine-tuning recipe for retrieval-augmented
    generation (RAG) which combines pre-trained parametric and non-parametric memory
    for language generation. ArXiv ID: 2005.11401""",

    """Title: Self-RAG: Learning to Retrieve, Generate, and Critique through Self-Reflection
    Authors: Asai, Wu, Wang, Sil, Hajishirzi
    Abstract: We introduce Self-RAG, a framework that trains a single LM to adaptively
    retrieve passages on demand, generate text informed by retrieved passages, and
    critique its own output using special reflection tokens. Self-RAG significantly
    outperforms existing RAG approaches and LLMs on multiple tasks. ArXiv ID: 2310.11511""",
]

sci_docs = DocumentLoader.from_texts(sample_scientific, "scientific", "demo_papers")
sci_chunks = chunker.chunk_documents(sci_docs, domain="scientific")
count = vs.add_documents("scientific", sci_chunks)
print(f"✅ Indexed {count} scientific chunks")

# Status check
print("\n📊 Index Status:")
for stat in vs.get_all_stats():
    print(f"  {stat['domain']:>12}: {stat['document_count']} chunks")

✅ Indexed 6 industrial chunks
✅ Indexed 3 recipe chunks
✅ Indexed 3 scientific chunks

📊 Index Status:
    industrial: 6 chunks
        recipe: 3 chunks
    scientific: 3 chunks


In [12]:
import sys

# Make sure the project root is at the FRONT of the path
PROJECT = '/content/drive/MyDrive/AI574_Multi_Domain_Agent/project'
if PROJECT not in sys.path:
    sys.path.insert(0, PROJECT)

# Verify the right modules are found
import agents.industrial_agent
print(f"✅ Found: {agents.industrial_agent.__file__}")

✅ Found: /content/drive/MyDrive/AI574_Multi_Domain_Agent/project/agents/industrial_agent.py


In [13]:
from orchestration.workflow_graph import build_workflow, run_query

workflow = build_workflow(llm=llm, vector_store=vs, index_builder=builder)
print("✅ Workflow compiled!")

# Test: Rockwell industrial query
result = run_query(workflow, "My PowerFlex 525 drive is showing fault F004, how do I fix it?")
print(f"\nDomain: {result['domain']}")
print(f"Confidence: {result['confidence']:.2f}")
print(f"\nResponse:\n{result['response']}")

✅ Workflow compiled!

Domain: industrial
Confidence: 0.99

Response:
Your PowerFlex 525 drive is showing fault code F004, which indicates an undervoltage condition.  Here's how to troubleshoot it based on the provided information from Rockwell Automation:

**SAFETY:** Disconnect and lockout/tagout all power sources before inspecting terminals. Wait 5 minutes for DC bus capacitors to discharge before servicing.

**Troubleshooting Steps:**

1. **Measure incoming line voltage:** Check the voltage at the drive terminals (R/L1, S/L2, T/L3). The voltage must be within the nameplate rating +/-10%.
2. **Check for voltage balance:** Ensure all three phases have voltage and that the imbalance is no more than 3%.
3. **Inspect input power terminal connections:**  Visually inspect the connections for looseness and tighten them to 1.4 Nm torque.
4. **Check DC bus voltage:** Review parameter A531 in the drive's settings to see the current DC bus voltage reading.
5. **Review fault history:**  Examine 

In [14]:
result = run_query(workflow, "What can I substitute for eggs in cookies?")
print(f"Domain: {result['domain']} | Confidence: {result['confidence']:.2f}")
print(f"\nResponse:\n{result['response']}")

Domain: recipe | Confidence: 0.95

Response:
This recipe suggests a few egg substitutes for your cookies:

* **Unsweetened Applesauce:** Use 1/4 cup per egg. Applesauce adds moisture and a slight sweetness, similar to eggs. 
* **Mashed Banana:** Use 1 mashed banana per egg. Banana will add moisture and a subtle banana flavor.
* **Aquafaba:** Use 3 tablespoons per egg. Aquafaba (the liquid from canned chickpeas) can act as a binder and create a similar texture to eggs when whipped. 


Let me know if you'd like to explore other substitutions or have more questions about this recipe!


In [15]:
# Test: Scientific query
result = run_query(workflow, "Summarize the key ideas behind RAG in NLP")
print(f"Domain: {result['domain']} | Confidence: {result['confidence']:.2f}")
print(f"\nResponse:\n{result['response']}")


Domain: scientific | Confidence: 0.95

Response:
## Retrieval-Augmented Generation (RAG) in NLP: A Summary

**Objective:**  To improve the performance of language models (LMs) on knowledge-intensive NLP tasks by incorporating external knowledge sources.

**Method:** RAG combines the strengths of LMs with external knowledge bases.  

* **Document 1 (Asai et al., 2023)** proposes Self-RAG, a framework where a single LM learns to retrieve relevant passages, generate text based on those passages, and even critique its own output using special reflection tokens. 
* **Document 2 (Lewis et al., 2020)** explores a general-purpose fine-tuning recipe for RAG that utilizes both parametric (e.g., embeddings) and non-parametric (e.g., dense passage retrieval) memory mechanisms to enhance language generation.

**Key Findings:**

* Both papers highlight the effectiveness of RAG in boosting LM performance on knowledge-intensive tasks.
* Self-RAG (Asai et al.) demonstrates significant performance gains

In [16]:
# Test: Ambiguous query (should trigger clarification)
result = run_query(workflow, "What temperature should I set to avoid failure?")
print(f"Domain: {result['domain']} | Confidence: {result['confidence']:.2f}")
print(f"Needs clarification: {result['needs_clarification']}")
print(f"\nResponse:\n{result['response']}")

Domain: clarify | Confidence: 0.60
Needs clarification: True

Response:
Are you asking about the temperature for cooking something, or for a machine or device?


# New Section